In [1]:
import os
import sys
import pandas as pd
import numpy as np
import quandl

In [2]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [3]:
%matplotlib inline

In [4]:
tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE']
start_date='2012-01-01'
end_date='2016-12-31'
returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'].pct_change())
                for ticker in tickers]))
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,-0.007503,-0.002322,-0.004411,0.023459,0.000020
2016-12-27,0.014213,0.002637,-0.011944,0.029015,0.000020
2016-12-28,0.000946,-0.006618,-0.005264,0.000957,0.000021
2016-12-29,-0.009040,-0.002101,0.000784,-0.023027,0.000018
2016-12-30,-0.019970,-0.012991,-0.004505,-0.004612,0.000020


In [5]:
r_hat = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
Sigma_hat = returns.rolling(window=250, min_periods=250).cov().stack().shift(1).unstack().dropna()

r_hat.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,0.000683,0.000219,-0.000660,-0.000074,0.000012
2016-12-27,0.000542,0.000150,-0.000706,-0.000124,0.000013
2016-12-28,0.000627,0.000179,-0.000691,-0.000023,0.000013
2016-12-29,0.000707,0.000215,-0.000664,-0.000052,0.000013
2016-12-30,0.000901,0.000302,-0.000598,0.000133,0.000013


In [8]:
returns = pd.read_csv(r'Y:/projects/cvxportfolio/data/returns.csv.gz', index_col=0, parse_dates=[0])
returns.rename(columns={"USDOLLAR": "_CASH"}, inplace=True)

volumes = pd.read_csv(r'Y:/projects/cvxportfolio/data/volumes.csv.gz', index_col=0, parse_dates=[0])
volumes.rename(columns={"USDOLLAR": "_CASH"}, inplace=True)

volumes["_CASH"] = 0.
volumes = volumes.iloc[1:]

sigmas = pd.read_csv(r'Y:/projects/cvxportfolio/data/sigmas.csv.gz', index_col=0, parse_dates=[0])
sigmas.rename(columns={"USDOLLAR": "_CASH"}, inplace=True)
sigmas["_CASH"] = 0.
sigmas = sigmas.iloc[1:]

In [9]:
w_b = pd.Series(index=returns.columns, data=1)
w_b['_CASH'] = 0.
w_b/=sum(w_b)
start_date=returns.index[1]
end_date=returns.index[-1]

In [10]:
simulated_hcost = helium.HoldingCost(borrow_costs=0.0001, dividends=0., gamma=1.)
simulated_tcost = helium.TransactionCost(half_spread=0.00005/2., nonlin_coef=-1., nonlin_power=1.5, asym_coef=0., sigmas=sigmas, volumes=volumes, gamma=1. )

In [18]:
import multiprocess
?multiprocess.Pool

In [19]:
pool = multiprocess.Pool(3)

In [24]:
??pool.map